**Pre-Trained ViT Model**

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/Train.zip'  # update with your zip file path
extract_to = '/content/drive/MyDrive'  # change if you want to unzip to a different folder

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Unzipped to:", extract_to)


Unzipped to: /content/drive/MyDrive


In [ ]:
# Install necessary libraries
!pip install -q transformers datasets torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -U transformers datasets

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score
import torch

# Step 1: Load image processor
image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

# Step 2: Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std),
])

# Step 3: Load dataset
dataset = ImageFolder(root="/content/drive/MyDrive/Train", transform=transform)

# Step 4: Convert to Hugging Face Dataset format
def transform_example(example):
    return {
        "pixel_values": example[0],
        "label": example[1],
    }

hf_dataset = Dataset.from_generator(lambda: map(transform_example, dataset))

# Step 5: Split dataset
hf_dataset = hf_dataset.train_test_split(test_size=0.1)
train_ds = hf_dataset["train"]
val_ds = hf_dataset["test"]

train_ds.set_format(type="torch")
val_ds.set_format(type="torch")

# Step 6: Load model
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=10,
    ignore_mismatched_sizes=True
)

# Step 7: Define metric function
def compute_metrics(preds, labels):
    pred_labels = np.argmax(preds, axis=1)
    acc = accuracy_score(labels, pred_labels)
    return {"accuracy": acc}

# Step 8: Training arguments (remove evaluation_strategy)
training_args = TrainingArguments(
    output_dir="./vit-results",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

# Step 9: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor  # still required
)

# Step 10: Train manually with evaluation per epoch
for epoch in range(int(training_args.num_train_epochs)):
    print(f"\nEpoch {epoch + 1}/{int(training_args.num_train_epochs)}")
    trainer.train()
    eval_result = trainer.evaluate()
    preds = np.argmax(eval_result['logits'], axis=1) if 'logits' in eval_result else None
    if preds is not None:
        acc = compute_metrics(preds, eval_result['label_ids'])
        print(f"Validation Accuracy: {acc['accuracy']:.4f}")
    else:
        print(f"Validation Accuracy: {eval_result.get('eval_accuracy', 'N/A'):.4f}")
    print(f"Validation Loss: {eval_result.get('eval_loss', 'N/A'):.4f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-4-282655c91aab>:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Epoch 1/5


Step,Training Loss
10,1.718200
20,0.760500
30,0.395300
40,0.175500
50,0.146000
60,0.088100
70,0.071500
80,0.058900
90,0.061300
100,0.052300


ValueError: Unknown format code 'f' for object of type 'str'

In [ ]:
# Prediction and custom accuracy
predictions = trainer.predict(val_ds)
acc = compute_metrics(predictions.predictions, predictions.label_ids)
print(f"Validation Accuracy: {acc['accuracy']:.4f}")


Validation Accuracy: 0.9974
